### 8.
Load the MNIST data, and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM classifier. Next, try to combine them into an ensemble that outperforms each individual classifier on the validation set, using soft or hard voting. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

In [1]:
from sklearn.datasets import fetch_openml
import  numpy as np

mnist = fetch_openml('mnist_784', version=1)

X, y = mnist["data"], mnist["target"].astype(np.uint8)

In [2]:
# separate into training, validation and test sets
X_train, X_valid, X_test, y_train, y_valid, y_test = X[:50000], X[50000:60000], X[60000:], y[:50000], y[50000:60000], y[60000:]

In [9]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

rnd_clf = RandomForestClassifier()
rnd_clf.fit(X_train, y_train)

ext_clf = ExtraTreesClassifier()
ext_clf.fit(X_train, y_train)

svc_clf = SVC()
svc_clf.fit(X_train, y_train)

SVC()

In [10]:
from sklearn.metrics import accuracy_score

for clf in (rnd_clf, ext_clf, svc_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.9695
ExtraTreesClassifier 0.9715
SVC 0.9785


In [ ]:
from sklearn.ensemble import VotingClassifier

voting_hard_clf = VotingClassifier(estimators=[('rf', rnd_clf), ('et', ext_clf), ('svc', svc_clf)],
                             voting='hard')

voting_soft_clf = VotingClassifier(estimators=[('rf', rnd_clf), ('et', ext_clf), ('svc', svc_clf)],
                             voting='soft')

voting_hard_clf.fit(X_train, y_train)
voting_soft_clf.fit(X_train, y_train)

In [ ]:
for clf in (rnd_clf, ext_clf, svc_clf, voting_hard_clf, voting_soft_clf):
    y_pred = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

in progress